In [1]:
import pandas as pd
import numpy as np
import os
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,accuracy_score, f1_score
import joblib

C:\Users\gshik\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gshik\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\gshik\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


## Importing data

In [2]:
Train = pd.read_csv('data/Train.csv', index_col=0)
Test = pd.read_csv('data/Test.csv', index_col=0)

In [3]:
Y_train =Train['is_returning_customer']
X_train = Train.drop(['customer_id', 'is_returning_customer'],axis = 1)

Y_test =Test['is_returning_customer']
X_test = Test.drop(['customer_id', 'is_returning_customer'],axis = 1)

## Initializing Models:

In [4]:
gbc = GradientBoostingClassifier()
rfc = RandomForestClassifier()

<Strong> Why Random Forest and GBM? </Strong>

1. The problem in hand is <Strong> Supervised Binary Calssification Problem </Strong>. Some of the state of art Classification Algorithms are:<br>
    1.1 SVM<br>
    1.2 Random Forest<br>
    1.3 GBM<br>
    1.4 XGB<br>
Given enough data and features all of these algorithms have been proved to perform better than Traditional Statistical Algorithms like Logistic and Naive Bayes.


2. Among these apart from SVM which is based on the results of single classifier, all the other 3 are ensemble algorithms making predictions based on the results of several classifiers and generally outperform a single classifier.


3. Random Forest is bagging algorithm and makes prediction based on unweighted samples whereas GBM & XGB are boosting algorithms that re-evaulates sample weights based on misclassification. Also all the trees in RF are independent and unweighted whereas in boosting all the trees are dependent on the prior trees and are weighted. 


4. Thus though RF can help to reduce Variance, Boosting reduces both Variance and Bias but however is more prone to overfitting due to continuos weight adjustment(can lead to extremly high weights for few samples which are actually noise)



5. It thus make sense to test the results across train and test set using any of the <strong> Bagging Algorithms (Random Forest in our case) </strong> and one of the <strong> Boosting Algorithms(GBM in our case)</strong>  and compare their Performance 

## Randomized grid Search for Random Forest  and GBM Classifier

In [ ]:
# setup the hyperparameter grid
param_dist = {
    'bootstrap': [True],
    'n_estimators': [100,200,300],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'random_state':[11]
}


# Instantiate the grid search model for Random Forest Classifier
randomized_search_rfc = RandomizedSearchCV(estimator = rfc, param_distributions = param_dist, 
                          n_iter=20, cv = 3, random_state=11)

# Fit it to the data
randomized_search_rfc.fit(X_train, Y_train)


# setup the hyperparameter grid
param_dist = {
    'n_estimators': [100,200,300],
    'max_depth': [1, 2, 3],
    'min_samples_leaf': [10, 20, 30],
    'min_samples_split': [5, 10, 20],
    'random_state':[11]
}

# Instantiate the grid search model for Gradien Boosting Classifier
randomized_search_gbc = RandomizedSearchCV(estimator = gbc, param_distributions= param_dist, 
                          n_iter = 20, cv = 3, random_state=11)


# Fit it to the data
randomized_search_gbc.fit(X_train, Y_train)



In [ ]:
print("\n")
print("----------- Best Parametrs for Random Forest Model are:-------------")
print("\n")

print(randomized_search_rfc.best_params_)

print("\n")
print("----------- Accuracy Score for above Random Forest classifier on Train Data:-------------")
print("\n")

print(randomized_search_rfc.best_score_)

print("\n")
print("----------- Best Parametrs for Gradient Boosting Model are:-------------")
print("\n")

print(randomized_search_gbc.best_params_)

print("\n")
print("----------- Accuracy Score for above Gradient Boosting classifier on Train Data:-------------")
print("\n")

print(randomized_search_gbc.best_score_)

## Best Parameters from Grid Search:
<br>
<strong> Best Parametrs for Random Forest Model are:</strong>
<br>
{'random_state': 11, 'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 20, 'max_depth': 7, 'bootstrap': True}

<br>
<br>
Accuracy Score for above Random Forest classifier on Train Data:
<br>

0.8316522741151166
<br>
<br>
<br>
<br>
<strong> Best Parametrs for Gradient Boosting Model are:</strong>
<br>

{'random_state': 11, 'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 30, 'max_depth': 3}

<br>
<br>
Accuracy Score for above Gradient Boosting classifier on Train Data:
<br>

0.8346321905872971

## Model Training and Validation

In [6]:
print("\n")
print("----------- Training the two models on the Best Paramters from Grid Search-------------")
print("\n")

gbc = GradientBoostingClassifier(random_state= 11, n_estimators= 300, min_samples_split= 5,\
                                 min_samples_leaf= 30, max_depth= 3)
rfc = RandomForestClassifier(random_state= 11, n_estimators= 100, min_samples_split=10,\
                                    min_samples_leaf= 20, max_depth= 7, bootstrap=True)

gbc.fit(X_train, Y_train)
rfc.fit(X_train, Y_train)
                             
print("\n")
print("----------- Getting Predictions from Models-------------")
print("\n")
gbc_class = gbc.predict(X_test)
gbc_prob = gbc.predict_proba(X_test)
gbc_prob = pd.DataFrame(gbc_prob, columns = ['Prob_0','Prob_1'])
gbc_class2 = gbc.predict(X_train)
gbc_prob2 = gbc.predict_proba(X_train)
gbc_prob2 = pd.DataFrame(gbc_prob2, columns = ['Prob_0','Prob_1'])

rfc_class = rfc.predict(X_test)
rfc_prob = rfc.predict_proba(X_test)
rfc_prob = pd.DataFrame(rfc_prob, columns = ['Prob_0','Prob_1'])
rfc_class2 = rfc.predict(X_train)
rfc_prob2 = rfc.predict_proba(X_train)
rfc_prob2 = pd.DataFrame(rfc_prob2, columns = ['Prob_0','Prob_1'])


print("\n")
print("----------- Model Performance on Validation Set-------------")
print("\n")

Performnace_DF = pd.DataFrame(columns = ["RF_train", "RF_test", "GBM_train", "GBM_test"],\
             index = ["RMSE", "Accuracy", "F1_Score"])

rmse = np.sqrt(mean_squared_error(Y_test, gbc_class))
rmse2 = np.sqrt(mean_squared_error(Y_train, gbc_class2))
Performnace_DF.loc["RMSE", "GBM_test"]=rmse
Performnace_DF.loc["RMSE", "GBM_train"]=rmse2

accuracy = accuracy_score(Y_test, gbc_class)
accuracy2 = accuracy_score(Y_train, gbc_class2)
Performnace_DF.loc["Accuracy", "GBM_test"]=accuracy
Performnace_DF.loc["Accuracy", "GBM_train"]=accuracy2

F1_Score = f1_score(Y_test, gbc_class)
F1_Score2 = f1_score(Y_train, gbc_class2)
Performnace_DF.loc["F1_Score", "GBM_test"]=F1_Score
Performnace_DF.loc["F1_Score", "GBM_train"]=F1_Score2

rmse = np.sqrt(mean_squared_error(Y_test, rfc_class))
rmse2 = np.sqrt(mean_squared_error(Y_train, rfc_class2))
Performnace_DF.loc["RMSE", "RF_test"]=rmse
Performnace_DF.loc["RMSE", "RF_train"]=rmse2

accuracy = accuracy_score(Y_test, rfc_class)
accuracy2 = accuracy_score(Y_train, rfc_class2)
Performnace_DF.loc["Accuracy", "RF_test"]=accuracy
Performnace_DF.loc["Accuracy", "RF_train"]=accuracy2

F1_Score = f1_score(Y_test, rfc_class)
F1_Score2 = f1_score(Y_train, rfc_class2)
Performnace_DF.loc["F1_Score", "RF_test"]=F1_Score
Performnace_DF.loc["F1_Score", "RF_train"]=F1_Score2

Performnace_DF['%diff_RF'] = (Performnace_DF.RF_train-Performnace_DF.RF_test)\
*100/Performnace_DF.RF_train
Performnace_DF['%diff_GBM'] = (Performnace_DF.GBM_train-Performnace_DF.GBM_test)\
*100/Performnace_DF.GBM_train

print("\n")
print("----------- Performance Comaprision across Train and Test for RF and GBM-------------")
print("\n")
Performnace_DF



----------- Training the two models on the Best Paramters from Grid Search-------------




----------- Getting Predictions from Models-------------




----------- Model Performance on Validation Set-------------




----------- Performance Comaprision across Train and Test for RF and GBM-------------




,RF_train,RF_test,GBM_train,GBM_test,%diff_RF,%diff_GBM
RMSE,0.407669,0.408919,0.403234,0.404763,-0.306443,-0.379205
Accuracy,0.833806,0.832786,0.837403,0.836167,0.122348,0.147539
F1_Score,0.505327,0.500426,0.538727,0.534568,0.969860,0.772021


## Observations from the Performance Results:

1. GBM performs better than RF on validation set on all parameters whether it is RMSE, Accuracy or F1 score
    

2. Moreover neither of the model is overfitting as can been seen from the % diff_RF/ % diff_GBM due to regularization.
    
<strong> Therefore we can conclude that GBM is better model than the RF </Strong>

In [7]:
# function to calculate ks and Capture Rate 

def KS_table_10(score, response):
    print('getting KS..')
    group = 10
    df = pd.DataFrame({'score': score, 'response' : response})
    df = df.sort_values(by = 'score', ascending = False)
    bin_size = len(score)/group
    rem = len(score) % group
    df['groups'] = list(np.repeat(range(1,rem+1),bin_size + 1)) + list(np.repeat(range(rem+1,11), bin_size))
    grouped = df.groupby('groups', as_index =False)
    agg = pd.DataFrame({'Total_Obs': grouped.count().response})
    agg['No.Res'] = grouped.sum().response
    agg['No.Non_Res'] = agg['Total_Obs'] - agg['No.Res']
    agg['min_pred'] = grouped.min().score
    agg['max_pred'] = grouped.max().score
    agg['pred_rr'] = grouped.mean().score
    agg['cum_no_res'] = agg['No.Res'].cumsum()
    agg['cum_no_non_res'] = agg['No.Non_Res'].cumsum()
    agg['percent_cum_res'] = agg['cum_no_res']/agg['cum_no_res'].max()
    agg['percent_cum_non_res'] = agg['cum_no_non_res']/agg['cum_no_non_res'].max()
    agg['KS'] = agg['percent_cum_res'] - agg['percent_cum_non_res']
    max_ks = agg['KS'].max()
    return(agg, max_ks)

Decile, KS = KS_table_10(gbc_prob['Prob_1'], Y_test)
print("\n")
print('KS for GBM is: ', KS)
print(Decile['percent_cum_res'][2]*100, '% of responders are captured in top 3 Deciles for with GBM.')
print("\n")

Decile, KS = KS_table_10(rfc_prob['Prob_1'], Y_test)
print("\n")
print('KS for Random Forest is: ', KS)
print(Decile['percent_cum_res'][2]*100, '% of responders are captured in top 3 Deciles for with Random Forest.')
print("\n")

print("\n")
print("----------- GBM is better even in terms of KS and Capture rate on validation set.-------------")
print("\n")

getting KS..


KS for GBM is:  0.4875215379894331
67.76078053481089 % of responders are captured in top 3 Deciles for with GBM.


getting KS..


KS for Random Forest is:  0.472980330576563
66.634545892556 % of responders are captured in top 3 Deciles for with Random Forest.




----------- GBM is better even in terms of KS and Capture rate on validation set.-------------




In [8]:
print("\n")
print("----------- Saving Final Model Files-------------")
print("\n")

joblib.dump(gbc,'model/GBM_Classifer.joblib')
joblib.dump(rfc,'model/RF_Classifer.joblib')



----------- Saving Final Model Files-------------




['model/RF_Classifer.joblib']